In [1]:
from config import set_project_root
set_project_root()

In [2]:
import pandas as pd
import numpy as np
from finance_byu.summarize import summary
import matplotlib.pyplot as plt
import seaborn as sns
from research.datasets import CRSP
import statsmodels.formula.api as smf

In [3]:
# Read in crsp monthly data

df = CRSP().df.copy()

df.head()

LOADING CLEAN FILE


,permno,date,cusip,shrcd,exchcd,ticker,shrout,vol,prc,ret
0,10000,1986-02-28,68391610,10.0,3.0,OMFGA,3680.0,828.0,3.25000,-0.257143
1,10000,1986-03-31,68391610,10.0,3.0,OMFGA,3680.0,1078.0,4.43750,0.365385
2,10000,1986-04-30,68391610,10.0,3.0,OMFGA,3793.0,957.0,4.00000,-0.098592
3,10000,1986-05-30,68391610,10.0,3.0,OMFGA,3793.0,1074.0,3.10938,-0.222656
4,10000,1986-06-30,68391610,10.0,3.0,OMFGA,3793.0,1069.0,3.09375,-0.005025


In [4]:
# In sample
start = np.datetime64('1929-01-01')
end = np.datetime64('1982-12-31')

df = df[(df['date'] >= start) & (df['date'] <= end)]

df = df.reset_index(drop=True)

df

,permno,date,cusip,shrcd,exchcd,ticker,shrout,vol,prc,ret
0,10006,1929-01-31,00080010,10.0,1.0,None,600.0,359.0,99.2500,0.011465
1,10006,1929-02-28,00080010,10.0,1.0,None,600.0,140.0,99.5000,0.002519
2,10006,1929-03-28,00080010,10.0,1.0,None,600.0,634.0,100.7500,0.027638
3,10006,1929-04-30,00080010,10.0,1.0,None,600.0,127.0,98.5000,-0.022333
4,10006,1929-05-31,00080010,10.0,1.0,None,600.0,107.0,94.0000,-0.045685
...,...,...,...,...,...,...,...,...,...,...
1226612,92946,1982-07-30,92922210,11.0,3.0,VYQT,1870.0,NaN,NaN,NaN
1226613,92946,1982-08-31,92922210,11.0,3.0,VYQT,1870.0,NaN,NaN,NaN
1226614,92946,1982-09-30,92922210,11.0,3.0,VYQT,1870.0,NaN,NaN,NaN
1226615,92946,1982-10-29,92922210,11.0,3.0,VYQT,1870.0,NaN,NaN,NaN


In [5]:
# Cleaning

df = df[['permno', 'ticker', 'date', 'prc', 'ret']].copy()

df['mdt'] = pd.to_datetime(df['date']).dt.strftime("%Y-%m")
df['month'] = pd.to_datetime(df['date']).dt.strftime("%m")

df.head()

,permno,ticker,date,prc,ret,mdt,month
0,10006,None,1929-01-31,99.25,0.011465,1929-01,01
1,10006,None,1929-02-28,99.50,0.002519,1929-02,02
2,10006,None,1929-03-28,100.75,0.027638,1929-03,03
3,10006,None,1929-04-30,98.50,-0.022333,1929-04,04
4,10006,None,1929-05-31,94.00,-0.045685,1929-05,05


In [6]:
df['ret_60_mean'] = df.groupby('permno')['ret'].rolling(59,59).mean().reset_index(drop=True)
df['ret_60_mean'] = df.groupby('permno')['ret_60_mean'].shift(-1)

df['xs_ret'] = df['ret'] - df['ret_60_mean']

def create_lags(df, lags, column='ret'):
    for lag in lags:
        df[f'{column}_lag_{lag}'] = df.groupby('permno')[column].shift(lag)
    return df

lags = list(range(1, 13)) + [24, 36]
df = create_lags(df, lags)

df = df.dropna()

df

,permno,ticker,date,prc,ret,mdt,month,ret_60_mean,xs_ret,ret_lag_1,...,ret_lag_5,ret_lag_6,ret_lag_7,ret_lag_8,ret_lag_9,ret_lag_10,ret_lag_11,ret_lag_12,ret_lag_24,ret_lag_36
405,10006,ACF,1962-07-31,67.5000,0.144068,1962-07,07,0.012450,0.131618,-0.012552,...,0.011986,0.087523,0.082661,0.043750,0.043478,-0.033613,0.006276,0.066964,-0.013514,0.064838
406,10006,ACF,1962-08-31,67.1250,0.003704,1962-08,08,0.013075,-0.009371,0.144068,...,-0.071672,0.011986,0.087523,0.082661,0.043750,0.043478,-0.033613,0.006276,-0.021918,0.056206
407,10006,ACF,1962-09-28,63.5000,-0.054004,1962-09,09,0.014700,-0.068704,0.003704,...,0.056985,-0.071672,0.011986,0.087523,0.082661,0.043750,0.043478,-0.033613,-0.142045,-0.067265
408,10006,ACF,1962-10-31,68.0000,0.070866,1962-10,10,0.018294,0.052572,-0.054004,...,-0.160000,0.056985,-0.071672,0.011986,0.087523,0.082661,0.043750,0.043478,-0.013245,-0.038462
409,10006,ACF,1962-11-30,72.8750,0.080882,1962-11,11,0.016951,0.063931,0.070866,...,-0.012552,-0.160000,0.056985,-0.071672,0.011986,0.087523,0.082661,0.043750,0.003356,-0.030000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226386,86239,CF,1971-07-30,20.5000,-0.052023,1971-07,07,0.020030,-0.072053,-0.082412,...,0.060000,0.025641,0.106383,0.004225,-0.003509,-0.080645,0.003846,0.026316,-0.044681,0.049351
1226387,86239,CF,1971-08-31,23.8750,0.174390,1971-08,08,0.019370,0.155020,-0.052023,...,0.000000,0.060000,0.025641,0.106383,0.004225,-0.003509,-0.080645,0.003846,-0.016854,0.025000
1226388,86239,CF,1971-09-30,22.4375,-0.060209,1971-09,09,0.012771,-0.072980,0.174390,...,0.184524,0.000000,0.060000,0.025641,0.106383,0.004225,-0.003509,-0.080645,-0.085714,0.021951
1226389,86239,CF,1971-10-29,21.7500,-0.030641,1971-10,10,0.013355,-0.043996,-0.060209,...,-0.082412,0.184524,0.000000,0.060000,0.025641,0.106383,0.004225,-0.003509,0.013750,0.090361


In [7]:
df['date']

405       1962-07-31
406       1962-08-31
407       1962-09-28
408       1962-10-31
409       1962-11-30
             ...    
1226386   1971-07-30
1226387   1971-08-31
1226388   1971-09-30
1226389   1971-10-29
1226390   1971-11-30
Name: date, Length: 466866, dtype: datetime64[ns]

In [8]:
# Define the formula for the regression model using the lagged columns
formula = 'xs_ret ~ ' + ' + '.join([f'ret_lag_{lag}' for lag in lags])
df['year'] = df['date'].dt.year
years = df['year'].unique()

# Initialize empty lists to store the results
coefs_list = []
t_stats_list = []

for year in years:
    # Filter data for the current year
    slice_df = df[df['year'] == year].copy()
    
    # Fit the OLS regression model using the formula syntax
    model = smf.ols(formula=formula, data=slice_df)
    result = model.fit()
    
    coefs = {i for i in result.params.items()}

    coefs_list.append(result.params.values)
    t_stats_list.append(result.tvalues.values)

# Create a DataFrame with the results
results_df = pd.DataFrame({
    'year': years,
    'coefficients': coefs_list,
    't_statistics': t_stats_list
})

# Display the resulting DataFrame
results_df


,year,coefficients,t_statistics
0,1962,"[0.014291534895395322, -0.1777979493884163, -0...","[8.708654563420978, -14.686217238091977, -22.2..."
1,1963,"[0.005861014100920688, -0.14235916338422355, 0...","[7.180279992136678, -15.345760622279828, 6.038..."
2,1964,"[0.008612267182878152, -0.04159104360801425, -...","[10.614977881241943, -4.422412688200836, -1.15..."
3,1965,"[0.008090735238825454, -0.003512412729277348, ...","[8.948588638731648, -0.3692780059662274, -2.13..."
4,1966,"[-0.009450620631102491, -0.06112806212268748, ...","[-10.464462029772378, -6.823402375267478, 5.60..."
5,1967,"[0.026108205548544757, -0.04447403091062493, -...","[21.456513138066125, -5.649196288966213, -0.55..."
6,1968,"[0.016582681292946835, -0.022663314055208945, ...","[14.397669847384723, -3.0863907731090343, -20...."
7,1969,"[-0.036553925044049855, -0.06100217302439423, ...","[-39.29058855289881, -8.255841333256303, -2.65..."
8,1970,"[-0.016682347896310193, 0.0010368505456116684,...","[-13.599628675985834, 0.13951796784337867, -2...."
9,1971,"[0.019634608077591223, -0.03743770614296083, -...","[19.53764476961992, -5.136871507591996, -6.638..."
